# CTSM-SP simulations at NEON sites

### *run_neon* doesn't work for satelite phenology (SP cases), so we'll create these cases from *'scratch'*

The `run_neon` script we've been using is very useful, but it simplifies a number of steps that users may want to take control of in their workflow.  We'll introduce how you can control aspects of your case setup and configuration here. For example, a deeper understanding of the model is need if your want to run a satelite phenology (SP) case an unsupported tower (non-NEON) site, a generic single point simulation, a regional domain, or for global case.

If you're new to running CTSM, this is a somewhat advanced tutorial.  We're planning on doing all of our examples in the workshop using `run_neon`, so the information in this tutorial may not be needed. That said, it illustrates some important features of how to run the model, how to get initial condition files, and more.

<br>

---

## In this tutorial

The tutorial has several components. Below you will find steps to: 
1. Create, setup, build, and run a CTSM-SP case at a NEON site from scratch
2. Locate history files

<div class="alert alert-block alert-warning">

<b>NOTICE:</b> If you're running this notebook through the NCAR JupyterHub login you need to be on a Cheyenne login node (NOT Casper).  

</div>

<h1> 1. Set up and run a simulation</h1>

### CTSM can be run in 4 simple steps.

1. **Create** a new case
  - *This step sets up a new simulation. It is the most complicated of these four steps because it involves making choices to set up the model configuration*

2. **Setup** the case 
  - *This step configures the model so that it can compile*

3. **Build** the executable
  - *This step compiles the model*

4. **Submit** your run to the batch queue
  - *This step submits the model simulation*
  
**NOTE:** We'll also customine our case with xml and namelist changes.

***
<h1> 1.1 create a new case</h1>

*Set up a new simulation*
***


<div class="alert alert-block alert-info">
    
<b>TIP:</b> If you're running on Cheyenne, creating a softlink to scratch in your home directory is helpful.  
    
You can create this with the following: `ln -s /glade/scratch/$USER ~/scratch`

</div>

## 1.1.1 Navigate to the source code directory

Your source code is in your home directory

In [ ]:
cd ~/CTSM/cime/scripts

# Change the 4-character NEON site below.
export neon_site="YELL"

## 1.1.1 Create a new case

In [ ]:
# then create a new case.
./create_newcase --case ~/scratch/NEON_cases/${neon_site}_sp_test \
    --res CLM_USRDAT \
    --compset I1PtClm51SpRs \
    --output-root ~/scratch/NEON_cases/ \
    --run-unsupported \
    --user-mods-dir NEON/${neon_site} 

***
The code above doesn't always render properly online.  Here's what the code actually says:

```
./create_newcase --case ~/scratch/NEON_cases/${neon_site}_sp_test \
    --res CLM_USRDAT \
    --compset I1PtClm51SpRs \
    --output-root ~/scratch/NEON_cases/ \
    --run-unsupported \
    --user-mods-dir NEON/${neon_site} 
```
***
### **./create_newcase**

<div class="alert alert-block alert-info">

<b>NOTE:</b> There is a lot of information that goes into creating a case.

You can learn more about the options by typing <i>./create_newcase --help</i> on the the command line or in a new code cell.

<b>We'll briefly go over some of the highlights here.</b>

</div>

---

### Required arguments to create a new case
There are 3 Required arguments Needed to create a new case.  These include 
1. `--case`, which specifies the *location* and *name* of the case being created
  - `~/scratch` = the alias to your scratch  directory
  - `NEON_cases` = the subdirectory we created to store your other cases
  - `${neon_site}_sp_test` = the name of the case you're creating
  - *Recommendation:* Use meaningful names, including model version, type of simulation, and any additional details to help you remember the configuration of this simulation
<br><br>
2. `--res` Defines the model *resolution*, or grid,
  - `CLM_USRDAT`, which is an option to get a case setup without having to define the grid resolution, yet. 
  - In global cases, the land model is commonly run at a nominal 1 degree `f09_g17` or 2 degree `f19_g17` resolution 
  - Using `./query_config --grids` provides a list of supported model resolutions
<br><br>  
3. `--compset` Defines the *component set* for your case, 
  - The Component set specifies the default configuration for the case which includes:
    - Component models (e.g. active vs. data vs. stub), 
    - Time period of simulations and forcing scenarios (e.g. 1850 vs 2000 vs. HIST) and 
    - Physics options (e.g. CLM5.1 vs CLM5.0). 
  - `I1PtClm51SpRs` is alias that actually describes a much longer set of components that are being used for this single point case. 
  - All CLM-only compsets start with *"I"*.
    - Using `./query_config --compsets clm` provides examples of other CLM compsets
<br>

*There are a few other **optional** flags used to create this new case that we'll briefly touch on here*
<br>

4. `--output-root`, which specifies the *location* of your run directory
  - `~/scratch/NEON_cases` = the subdirectory we already created 
<br> 

5. `--user-mods-dirs` sets up the configuration of the case with a user modification directory that defines the location of the site.
  - `NEON/FATES/${neon_site}` has files that format of the history files and a few other custom settings for your case. 
  - This inludes setting some of .xml variables and name list settings correctly for a FATES simulation in a NEON single point case. 
<br>

6. `--run-unsupported` avoids error using compsets are not scientifically supported 

<div class="alert alert-block alert-info" markdown='1'>

<b>NOTE:</b> You may notice an error about project codes when you create your case. The project code isn't important for these simulations. But you may need to change this if you're running on Cheyenne.

</div>

***

<h2> 1.2 Customize your SP case </h2>

*Modifications for an SP case*

### 1.2.1 Move to your SP case directory


In [ ]:
cd ~/scratch/NEON_cases/${neon_site}_sp_test

**Background**

For an SP case, we want to run NEON sites for a bit to let soil water and temperatures the equilibrate before comparing to observations. Luckily, this only takes a few years (in most cases).

**What years is the model set up to run over by default?**  We can look by using `./xmlquery`

In [ ]:
./xmlquery DATM_YR_START DATM_YR_ALIGN DATM_YR_END

Most NEON sites we have 4 complete years of data to work with (2018-2021). 

If this is true for your site, we can use xml commands to modify our case


In general, we want to run for twice as many years as we have NEON observations.  
- CLM will just cycle over the meterological data that NEON provides.  
  - e.g. data from 2018-2021 will be used for 2014-2017 and then again for 2018-2021
- This will allow the second cycle of our our simulation to end with the same dates over which we have NEON observations, making the comaprision with observations easier. 

### 1.2.2 xml changes to modify env_run options
xml changes are one way you can control your case to customize the simulation
- We won't cover this here, but other tutorials cover more about xml changes.

**NOTE** `RUN_STARTDATE` is totally arbitrary. It's just the model date when the simulation starts. 
- xml variables like `DATM_YR_START` and `DATM_YR_ALIGN` determine the data that's being read into for your simulation. 

In [ ]:
./xmlchange RUN_STARTDATE=2014-01-01 
./xmlchange STOP_N=8  

### 1.2.3 Modify user_nl_clm
We also won't discuss namelist modificaiton here, but they are another way to customize your case.

Specifically, we're using usermod directories to write out high frequency history variables for your case in SP mode.
- **Remember** beyond simulating GPP, simulations in SP mode don't have a carbon cycle.  The variable for GPP in SP mode is called `FPSN`, which has different units than GPP.

You can look at the variables we'll write out on the `h1` history files (30 minute frequency on daily files)

In [ ]:
cat user_nl_clm | tail -1

## 1.3 Setup and build your case

*This step configures the model so that it can then compile* 

### 1.3.1 case.setup
The `./case.setup` script:
1. configures the model
2. creates files to modify input data and run options

In [ ]:
./case.setup

***
<h3> 1.3.2 Build the executable  </h3>

*This step compiles the model*

It also takes a long time, so be patient.
***

The `./case.build` script:
1. Checks input data
2. Creates a build/run directory with model executable and namelists

This takes some time, and will throw a bunch of errors and WARNINGS... 

--- **don't worry, be patient, and just give it time**   ---


In [ ]:
qcmd -- ./case.build


---

**When the build completes successfully you'll see a notice that the `MODEL BUILD HAS FINISHED SUCCESSFULLY`**


You can read on, but before executing any code blocks in the notebook **wait for the model to build.**
This can take a while.

---

<h3> 1.4 Submit your case  </h3>

Now you're ready to submit the case!

*This step submits the model simulation*
- You'll also be downloading all the meterological data from NEON for your site, this also takes a little time and prints lots of information to the screen
***

In [ ]:
./case.submit

When you submit a job, you will see confirmation that it successfully submitted:

### Congratulations! 
### You've created and submitted a single point NEON run from scratch.

Next, you will probably want to check on the status of your jobs.

<div class="alert alert-block alert-info" markdown="1">

<b>TIP:</b> This is dependent on the scheduler that you're using. 

<ul>
<li>Cheyenne uses <b>PBS</b> where status is checked with <i>qstat -u $USER</i></li>
<li>This is also enabled in the cloud for you, try it in the code block below</li></ul>

If you want to stop the simulation, you can do so with qdel here (or on Cheyenne).
<ul>
<li> Find your Job ID after typing <i>qstat</i> </li>
<li> Type <i>qdel {Job ID}</i> </li>
</ul>
</div>

In [ ]:
qstat -u $USER

---
Once your jobs are complete (or show the 'C' state under the 'Use' column, which means complete), we can check the CaseStatus file to ensure there were no errors and it completed successfully.  To do this, we'll 'tail' the end of the CaseStatus file:

In [ ]:
tail  ~/scratch/NEON_cases/${neon_site}_sp_test/CaseStatus

You should see several lines, with the middle one saying 'case.run success'.  

Before that you'll see notifications about xml changes, case.setup, and case.submit, and case.run

<div class="alert alert-success">
<strong>Congratulations!</strong> 
    
You've created a CTSM case for the NEON tower you selected.

We'll build on these basics in additional tutorials to customize your simulations.
</div>


****


# 2 Create a parallel BGC case for comparison
This is an optional example if you want to compare your SP case with a BGC case at the same site.
- Your `base-case` is the very first case you ran in the GettingStarted tutorial.  
- We'll use this to create a clone for a BGC experiment that doesn't have to rebuild.

In [ ]:
export base_case='KONZ'  # This can be the very first case you ran in the GettingStarted tutorial 
cd ~/CTSM/tools/site_and_regional/
./run_neon.py --neon-sites $neon_site  \
   --base-case ~/scratch/NEON_cases/$base_case \
   --output-root ~/scratch/NEON_cases 

## 2.1 Now check to see how your cases are different.
First we'll look at the `env_run.xml` files

In [ ]:
cd ~/scratch/NEON_cases/${neon_site}_sp_test
diff env_run.xml ../${neon_site}.transient/env_run.xml

Some of these differences are casued by the compset and our xml changes: 
- 2000_DATM vs. HIST_DATM provides constant vs. transient CO2
- SP vs. BGC
- time period of the simulation.

You can use the `diff` command to compare differences in the configuration of your SP and BGC case (hint look in CaseDocs):
- lnd_in
- datm.streams.xml

***
# 3. Create a Clone
<h4> This step is optional, but provides helpful information </h4>
    
Creating and building a new case is slow.  

Cloning cases can speed this up!

<h2>3.1 Create a clone </h2>

- Clones use the same resolution, compset, and output root.
- Clones can also share the same executable (build), which can save time!

<div class="alert alert-block alert-warning" markdown="1">

<b>WARNING:</b> If you're making code modifications be careful using clones that share the same build. The example below is a good way to run the same model configuration at different NEON sites.

</div>



In [ ]:
# Move back to your source code directory
cd ~/CTSM/cime/scripts

# Change the 4-character new NEON site you want to run.
export neon_site2="TEAK"

In [ ]:
# then create a cloned case.
./create_clone --case ~/scratch/NEON_cases/${neon_site2}_sp_test \
               --clone ~/scratch/NEON_cases/${neon_site}_sp_test \
               --user-mods-dirs ~/CTSM/cime_config/usermods_dirs/NEON/${neon_site2} \
               --keepexe


### **./create_clone**

<div class="alert alert-block alert-info">

<b>NOTE:</b> There is a lot of information that goes into creating a clone.

You can learn more about the options by typing <i>./create_clone --help</i> on the the command line or in a new code cell.

<b>We'll briefly go over some of the highlights here.</b>

</div>


### Required arguments to create a clone
There are 2 required arguments needed to create a clone.  These include: 
1. `--case`, this is the path and name of the new case your cloning.
<br><br>
2. `--clone`, this is the path and name of the existing case you want to clone.
<br><br>  
### Additional information we provided here were: 
3. `--user-mods-dirs` as before, this sets up the configuration of the case with a user modification directory that defines the location of the new site we're running, but it requires the full path to the `user-mods-dir` you want to use.
<br><br>  
4. `--keepexe` Point to original build from the case being cloned.

---

<h2> 3.2 Customize the clone? </h2>

*Because you created a clone and point to an existing build we can skip the `case.setup` and `case.build` steps* 
***

### 3.2.1 Move to your case directory and compare env_run

In [ ]:
cd ~/scratch/NEON_cases/${neon_site2}_sp_test
diff env_run.xml ../${neon_site}_sp_test/env_run.xml

### 3.2.2 Make xml changes
In the example above the **TEAK** data record only starts in 2019, meaning we have 3 years of meterological data to cycle through (2019-2021)
- CLM will cycle through the observational record for data that's available.
- If you want the observational record to line up correctily for sites that AREN'T standard (2018-2021), you may have made some custom XML changes
- Let's look at some particulars of time 

In [ ]:
./xmlquery RUN_STARTDATE STOP_OPTION STOP_N DATM_YR_START DATM_YR_ALIGN DATM_YR_END

For the **TEAK** example, if we start in 2014 our observational record will not align with the model years being simulated.

Uncomment out the code below to start in 2013 and cycle through the datm record three times.
- **NOTE** You may have to modify things differently for particular NEON sites

In [ ]:
#./xmlchange RUN_STARTDATE=2013-01-01 
#./xmlchange STOP_N=9  

***
<h2> 3.3 Submit the case </h2>

*Because you created a clone and point to an existing build we can skip the `case.setup` and `case.build` steps* 
***

- As before, you'll be downloading all the meterological data, which takes a little time and prints lots of information to the screen
***


In [ ]:
./case.submit

<div class="alert alert-success">
<strong>Congratulations!</strong> 
    
You've created and run a clone Fof CLM- for the NEON tower you selected.
</div>


You can track progress on your run using steps outlined in part #2 of this tutorial.
You can also work on visualizing your results for different sites using visualization code provided



If you're developing this tutorial:
Before saving and pushing this code to github go to `Kernel` and `Restart kernel and clear all outputs...`

In [ ]:
ls -lrt ~/scratch/NEON_cases/${neon_site2}_sp_test/run/ | tail -10